In [111]:
"""
Dataset extracted from Songsterr using selenium
Top 500 Drum Scores
Download as GP files
File conversion should be from GP to Midi or from GP to Music XML


"""

'\nDataset extracted from Songsterr using selenium\nTop 500 Drum Scores\nDownload as GP files\nFile conversion should be from GP to Midi or from GP to Music XML\n\n\n'

In [1]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor
from selenium.common.exceptions import InvalidSessionIdException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import asyncio
import time
import os

In [113]:
drum_score_folder = "/Users/jnaran/Desktop/SongsterrAPI/Drum_Scores"
sign_in_url = "https://www.songsterr.com/a/wa/signin"

In [114]:
#Define chrome_options
chrome_options = webdriver.ChromeOptions()
options = ['--headless', '--no-sandbox','--disable-dev-shm-usage']

for option in options:
  chrome_options.add_argument(option)

chrome_options.add_experimental_option("prefs",{
    "download.default_directory": drum_score_folder,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
    
})

In [115]:
links_driver = webdriver.Chrome(options=chrome_options)
links_driver.get("https://songsterr.com/?inst=drum")

In [116]:
scroll_down = """
var searchWrap = document.getElementById('search-wrap');
searchWrap.scrollTop = searchWrap.scrollHeight;
"""
rect_click = """
    arguments[0].dispatchEvent(new PointerEvent ('pointerdown', {
    bubbles: true,
    cancelable: true,
    pointerId: 1,
    pointerType: 'mouse',
    isPrimary: true
}));
"""

In [117]:
links = links_driver.find_elements(By.CSS_SELECTOR, "div[data-list='songs'] a[data-song]")

In [118]:
def get_links(links_driver,scroll_down):
    link_init = 0
    link_now = -1

    links = links_driver.find_elements(By.CSS_SELECTOR,"div[data-list='songs'] a[data-song]")  # initialize links
    
    while link_init != link_now:
        link_init = len(links)

        links_driver.execute_script(scroll_down)

        time.sleep(1)
        
        links = links_driver.find_elements(By.CSS_SELECTOR,"div[data-list='songs'] a[data-song]")

        link_now = len(links)

    return links

In [119]:
def urls(links):
    hrefs = []

    for link in links:
        href = link.get_attribute('href')
        hrefs.append(href)
        
    return hrefs


In [120]:
def return_links(links_driver,scroll_down):
    links = get_links(links_driver,scroll_down)
    hrefs = urls(links)
    return hrefs

In [121]:
songsterr_urls = return_links(links_driver,scroll_down)

In [122]:
#Initializing driver to extract scores
score_driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(score_driver, 10)

In [123]:
def action_click(element,driver):
    actions = ActionChains(driver)
    actions.move_to_element(element)
    actions.click()
    actions.perform()

In [124]:
def login(login_driver):

    login_keys = ['drumioai@gmail.com','M@)5eVzXSU:p)zs']

    username = login_driver.find_element(By.CSS_SELECTOR, "input[type='email']")
    password = login_driver.find_element(By.CSS_SELECTOR, "input[type='password']")

    username.send_keys(login_keys[0])
    password.send_keys(login_keys[1])

    login_button = login_driver.find_element(By.CSS_SELECTOR, "#signin")
    login_button.click()

    return login_driver

In [ ]:
def login_check(score_driver,timeout=10):
    counter = 0
    while score_driver.current_url != "https://www.songsterr.com/":
        time.sleep(1)
        counter += 1
        if counter > timeout:
            raise Exception("Login timeout")
   

In [126]:
def get_gp_files(score_driver,songsterr_urls):

    for url in songsterr_urls:
        score_driver.get(sign_in_url)

        #Logging in
        login(score_driver)
        login_check(score_driver)

        score_driver.get(url)

        b1 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "button#control-editor.Co02os")))
        b1.click()
        
        b2 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'rect.Bz2q3')))
        action_click(b2,score_driver)

        b3 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"button.z44l.z4xx[type='submit']")))
        b3.click()
        
        b4 = wait.until(EC.element_to_be_clickable((By.ID,"control-submitrevision")))
        b4.click()

        b5 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.Bta3n.Btaon")))
        b5.click()
    

In [138]:
get_gp_files(score_driver,songsterr_urls)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.Bta2eh > div.Bta3n.Btaon"}
  (Session info: chrome-headless-shell=121.0.6167.184); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000103222814 chromedriver + 4040724
1   chromedriver                        0x000000010321aa18 chromedriver + 4008472
2   chromedriver                        0x0000000102e8d870 chromedriver + 284784
3   chromedriver                        0x0000000102ed1080 chromedriver + 561280
4   chromedriver                        0x0000000102f0b048 chromedriver + 798792
5   chromedriver                        0x0000000102ec574c chromedriver + 513868
6   chromedriver                        0x0000000102ec6044 chromedriver + 516164
7   chromedriver                        0x00000001031e7a3c chromedriver + 

In [127]:
def main(songsterr_urls):
    with ThreadPoolExecutor(max_workers=len(songsterr_urls)) as executor:
        executor.map(get_gp_files,songsterr_urls)

In [ ]:
main(songsterr_urls)

In [128]:
songsterr_urls = return_links(links_driver,scroll_down)
get_gp_files(songsterr_urls)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[contains(@class, 'Bta3n') and contains(@class, 'Btaon') and contains(text(), 'Download your changes as a file')]"}
  (Session info: chrome-headless-shell=121.0.6167.184); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000103222814 chromedriver + 4040724
1   chromedriver                        0x000000010321aa18 chromedriver + 4008472
2   chromedriver                        0x0000000102e8d870 chromedriver + 284784
3   chromedriver                        0x0000000102ed1080 chromedriver + 561280
4   chromedriver                        0x0000000102f0b048 chromedriver + 798792
5   chromedriver                        0x0000000102ec574c chromedriver + 513868
6   chromedriver                        0x0000000102ec6044 chromedriver 